<a href="https://colab.research.google.com/github/SisekoC/My-Notebooks/blob/main/py_algotrading_ema_crossover_grid_search_batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmic Trading Model for Exponential Moving Average Crossover Grid Search Batch Mode Using Colab
### David Lowe
### June 25, 2020

NOTE: This script is for learning purposes only and does not constitute a recommendation for buying or selling any stock mentioned in this script.

SUMMARY: The purpose of this project is to construct and test an algorithmic trading model and document the end-to-end steps using a template.

INTRODUCTION: This algorithmic trading model examines a series of exponential moving average (MA) models via a grid search methodology. When the fast moving-average curve crosses above the slow moving-average curve, the strategy goes long (buys) on the stock. When the opposite occurs, we will exit the position.

The grid search methodology will search through all combinations between the two MA curves. The faster MA curve can range from 5 days to 20 days, while the slower MA can range from 10 days to 50 days. Both curves use a 5-day increment.

ANALYSIS: This is the Google Colab version of the iPython notebook posted on June 17, 2020. The script will save all output for each stock into a text file and on a Google Drive path. The Colab script contains an example of processing 100 different stocks in one batch.

CONCLUSION: Please refer to the individual output file for each stock.

Dataset ML Model: Time series analysis with numerical attributes

Dataset Used: Quandl

An algorithmic trading modeling project generally can be broken down into about five major tasks:

1. Prepare Environment
2. Acquire and Pre-Process Data
3. Develop Strategy and Train Model
4. Back-test Model
5. Evaluate Performance

## Task 1. Prepare Environment

In [ ]:
!pip install python-dotenv PyMySQ

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
import sys
import smtplib
import numpy as np
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dotenv import load_dotenv

In [ ]:
# Specify the moving average parameters for the trading strategy
initial_capital = 100000
fast_ma_min = 5
fast_ma_max = 20
slow_ma_min = 10
slow_ma_max = 50
ma_increment = 5
min_ma_gap = 5

# The number of extra days of data we need (usually equals to the larger of slow_ema or slow_sma)
extra_days_data = slow_ma_max

model_start_date = datetime(2015, 1, 1)
print("Starting date for the model:", model_start_date)
stock_start_date = model_start_date - timedelta(days=int(extra_days_data*1.5)) # Need more pricing data to calculate moving averages

model_end_date = datetime.now()
model_end_date = datetime(2020, 8, 3)
print("Ending date for the model:", model_end_date)

Starting date for the model: 2015-01-01 00:00:00
Ending date for the model: 2020-08-03 00:00:00


In [ ]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the verbose flag to print detailed messages for debugging (setting True will activate!)
verbose_models = True
verbose_graphs = True
verbose_portfolios = True

# Set up the sendNotification flag to send progress emails (setting True will send emails!)
notifyStatus = True

# Set up the parent directory location for loading the dotenv files
useColab = True
if useColab:
    # Mount Google Drive locally for storing files
    from google.colab import drive
    drive.mount('/content/gdrive')
    gdrivePrefix = '/content/gdrive/My Drive/Colab_Downloads/'
    env_path = '/content/gdrive/My Drive/Colab Notebooks/'
    dotenv_path = env_path + "python_script.env"
    load_dotenv(dotenv_path=dotenv_path)

# Set up the dotenv file for retrieving environment variables
useLocalPC = True
if useLocalPC:
    env_path = "C:/Users/cubas/OneDrive/Documents/Github Projects/4 Quantitative Finance Projects/py_algotrading_ema_crossover_grid_search_batch_colab"
    dotenv_path = env_path + "python_script.env"
    load_dotenv(dotenv_path=dotenv_path)

# Configure the plotting style
plt.style.use('seaborn')

# Set Pandas options
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 140)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<ipython-input-14-446de732cf01>:31: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


## Task 2. Acquire and Pre-Process Data

In [ ]:
pip install quandl

In [ ]:
# Check and see whether the API key is available
import quandl
quandl.ApiConfig.api_key = 'AywUrfKTvTTsCiRfWoWp'
quandl_key = quandl.ApiConfig.api_key
if quandl_key is None: sys.exit("API key for Quandl not available. Script Processing Aborted!!!")

In [ ]:
import requests
import json
import pandas as pd

def task2_acquire_process_data(stock_symbol, stock_start_date, model_end_date, quandl_key):
    start_date_string = stock_start_date.strftime('%Y-%m-%d')
    end_date_string = model_end_date.strftime('%Y-%m-%d')
    quandl_url = f"https://www.quandl.com/api/v3/datatables/SHARADAR/SEP.json?date.gte={start_date_string}&date.lte={end_date_string}&ticker={stock_symbol}&api_key={quandl_key}"

    print(f'Fetching equity data from {start_date_string} to {end_date_string}')

    try:
        response = requests.get(quandl_url)
        response.raise_for_status()  # Raise an error for bad status codes
        quandl_dict = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

    stock_quandl = pd.DataFrame(quandl_dict['datatable']['data'])
    print(f"{len(stock_quandl)} data points retrieved from the API call.")

    stock_quandl.columns = ['ticker', 'date', 'open', 'high', 'low', 'close', 'volume', 'dividend', 'closeunadj', 'lastupdated']
    stock_quandl['date'] = pd.to_datetime(stock_quandl['date'])  # Convert 'date' column to datetime
    stock_quandl.set_index('date', inplace=True)
    stock_quandl = stock_quandl.sort_index(ascending=True)

    return stock_quandl


## Task 3. Develop Strategy and Train Model

In [ ]:
import pandas as pd

def task3_develop_strategy(stock_quandl, verbose=False):
    # Select the data source and pricing columns to use for modeling
    model_template = stock_quandl.loc[:, ['open', 'close']]

    # Set up the standard column name for modeling
    model_template.rename(columns={'open': 'open_price', 'close': 'close_price'}, inplace=True)
    if verbose:
        model_template.info(verbose=True)

    # Initialize columns
    model_template['ma_change'] = 0  # Assuming you have a way to calculate this
    model_template['trade_signal'] = 0
    model_template['signal_change'] = 0
    model_template['entry_exit'] = 0

    def trading_ma_crossover(model):
        waitfor_first_entry = True
        for x in range(len(model)):
            if model['ma_change'].iloc[x] > 0:
                model.loc[x, 'trade_signal'] = 1  # trade_signal = 1 means we should take a long position
            else:
                model.loc[x, 'trade_signal'] = 0  # trade_signal = 0 means we should take a flat position
            if x != 0:
                model.loc[x, 'signal_change'] = model['trade_signal'].iloc[x] - model['trade_signal'].iloc[x-1]
                if waitfor_first_entry and (model['signal_change'].iloc[x-1] == 1):
                    model.loc[x, 'entry_exit'] = model['signal_change'].iloc[x-1]
                    waitfor_first_entry = False
                elif (not waitfor_first_entry) and (model['signal_change'].iloc[x-1] != 0):
                    model.loc[x, 'entry_exit'] = model['signal_change'].iloc[x-1]

    trading_ma_crossover(model_template)
    return model_template

import numpy as np

def generate_trading_models(model_template, slow_ma_min, slow_ma_max, fast_ma_min, fast_ma_max, ma_increment, min_ma_gap, model_start_date, model_end_date, verbose=False):
    model_collection = {}
    serial_number = 1
    for slow_ma in range(slow_ma_min, slow_ma_max + 1, ma_increment):
        for fast_ma in range(fast_ma_min, fast_ma_max + 1, ma_increment):
            if (slow_ma - fast_ma) < min_ma_gap:
                break
            model_name = f'SMA_{str(serial_number).zfill(3)}_SlowMA_{str(slow_ma).zfill(2)}_FastMA_{str(fast_ma).zfill(2)}'
            serial_number += 1
            trading_model = model_template.copy()
            trading_model['fast_ma'] = trading_model['close_price'].ewm(span=fast_ma).mean()
            trading_model['slow_ma'] = trading_model['close_price'].ewm(span=slow_ma).mean()
            trading_model['ma_change'] = trading_model['fast_ma'] - trading_model['slow_ma']
            trading_model['trade_signal'] = np.zeros(len(trading_model))
            trading_model['signal_change'] = np.zeros(len(trading_model))
            trading_model['entry_exit'] = np.zeros(len(trading_model))
            trading_model = trading_model[model_start_date:model_end_date]
            trading_ma_crossover(trading_model, 'ma_change')
            model_collection[model_name] = trading_model.copy()

    print(f"{serial_number - 1} models added to the trading model collection.")
    print()

    # List the entry/exit points for each model
    for key in model_collection:
        if verbose:
            print(f'List the signal change and entry/exit points for {key}')
            print(model_collection[key][(model_collection[key].signal_change != 0) | (model_collection[key].entry_exit != 0)])
            print()

    return model_collection

## Task 4. Back-test Model

In [ ]:
import pandas as pd

def task4_back_test_model(model_collection, initial_capital, verbose=False):
    def trading_portfolio_generation(initial_capital, trading_model):
        # Construct a portfolio to track the transactions and returns
        portfolio = pd.DataFrame(index=trading_model.index, columns=['trade_action', 'qty_onhand', 'cost_basis', 'sold_transaction', 'gain_loss', 'cash_onhand', 'position_value', 'total_position', 'accumu_return'])
        portfolio.iloc[0]['trade_action'] = 0
        portfolio.iloc[0]['qty_onhand'] = 0
        portfolio.iloc[0]['cost_basis'] = 0.00
        portfolio.iloc[0]['sold_transaction'] = 0.00
        portfolio.iloc[0]['gain_loss'] = 0.00
        portfolio.iloc[0]['cash_onhand'] = initial_capital
        portfolio.iloc[0]['position_value'] = 0.00
        portfolio.iloc[0]['total_position'] = initial_capital
        portfolio.iloc[0]['accumu_return'] = portfolio.iloc[0]['total_position'] - initial_capital
        recent_cost = 0

        # The conditional parameters below determine how the trading strategy will be carried out
        for i in range(1, len(portfolio)):
            if (trading_model.iloc[i]['entry_exit'] == 1) and (portfolio.iloc[i-1]['qty_onhand'] == 0):
                portfolio.loc[i, 'trade_action'] = 1
                portfolio.loc[i, 'qty_onhand'] = portfolio.iloc[i-1]['qty_onhand'] + portfolio.loc[i, 'trade_action']
                portfolio.loc[i, 'cost_basis'] = trading_model.iloc[i]['open_price'] * portfolio.loc[i, 'trade_action']
                portfolio.loc[i, 'sold_transaction'] = 0.00
                portfolio.loc[i, 'gain_loss'] = 0.00
                portfolio.loc[i, 'cash_onhand'] = portfolio.iloc[i-1]['cash_onhand'] - portfolio.loc[i, 'cost_basis']
                recent_cost = trading_model.iloc[i]['open_price'] * portfolio.loc[i, 'trade_action']
                if verbose: print('BOUGHT QTY:', portfolio.loc[i, 'trade_action'], 'on', portfolio.index[i], 'at the price of', trading_model.iloc[i]['open_price'])
            elif (trading_model.iloc[i]['entry_exit'] == -1) and (portfolio.iloc[i-1]['qty_onhand'] > 0):
                portfolio.loc[i, 'trade_action'] = -1
                portfolio.loc[i, 'qty_onhand'] = portfolio.iloc[i-1]['qty_onhand'] + portfolio.loc[i, 'trade_action']
                portfolio.loc[i, 'cost_basis'] = 0.00
                portfolio.loc[i, 'sold_transaction'] = trading_model.iloc[i]['open_price'] * portfolio.loc[i, 'trade_action'] * -1
                portfolio.loc[i, 'gain_loss'] = (recent_cost + (trading_model.iloc[i]['open_price'] * portfolio.loc[i, 'trade_action'])) * -1
                portfolio.loc[i, 'cash_onhand'] = portfolio.iloc[i-1]['cash_onhand'] + portfolio.loc[i, 'sold_transaction']
                recent_cost = 0.00
                if verbose: print('SOLD QTY:', portfolio.loc[i, 'trade_action'], 'on', portfolio.index[i], 'at the price of', trading_model.iloc[i]['open_price'])
            else:
                portfolio.loc[i, 'trade_action'] = 0
                portfolio.loc[i, 'qty_onhand'] = portfolio.iloc[i-1]['qty_onhand']
                portfolio.loc[i, 'cost_basis'] = portfolio.iloc[i-1]['cost_basis']
                portfolio.loc[i, 'sold_transaction'] = 0.00
                portfolio.loc[i, 'gain_loss'] = 0.00
                portfolio.loc[i, 'cash_onhand'] = portfolio.iloc[i-1]['cash_onhand']
            portfolio.loc[i, 'position_value'] = trading_model.iloc[i]['close_price'] * portfolio.loc[i, 'qty_onhand']
            portfolio.loc[i, 'total_position'] = portfolio.loc[i, 'cash_onhand'] + portfolio.loc[i, 'position_value']
            portfolio.loc[i, 'accumu_return'] = portfolio.loc[i, 'total_position'] - initial_capital

        return portfolio

    portfolio_collection = {}

    # Build dataframe for reporting model performance summary
    performance_summary = pd.DataFrame(columns=['model_name', 'return_value', 'return_percent'])

    for key in model_collection:
        portfolio_collection[key] = trading_portfolio_generation(initial_capital, model_collection[key])
        if initial_capital != 0:
            return_percentage = portfolio_collection[key].accumu_return[-1] / initial_capital * 100
        else:
            return_percentage = None
        performance_summary = performance_summary.append({'model_name': key, 'return_value': portfolio_collection[key].accumu_return[-1],
                                                          'return_percent': return_percentage}, ignore_index=True)
    print()

    # Display the model performance summary
    performance_summary.sort_values(by=['return_value'], inplace=True, ascending=False)
    print(performance_summary)
    print()

    # Display the transactions from the top model
    top_model = performance_summary.iloc[0]['model_name']
    print(f'The transactions from the top model {top_model}:')
    print(portfolio_collection[top_model][portfolio_collection[top_model].trade_action != 0])
    print()

    return portfolio_collection

## Task 5. Evaluate Performance

In [ ]:
import pandas as pd

def task5_evaluate_performance(stock_quandl, model_collection, portfolio_collection, initial_capital):
    # Calculate the stock's performance for a long-only model
    long_only_model = stock_quandl[model_start_date:model_end_date]
    long_only_return = long_only_model.iloc[-1]['close'] - long_only_model.iloc[0]['open']
    print('The performance of the long-only model from day one is: $%.2f' % long_only_return)
    print()

    best_model = ''
    best_return = 0
    for key in portfolio_collection:
        if portfolio_collection[key]['accumu_return'][-1] > best_return:
            best_model = key
            best_return = portfolio_collection[best_model]['accumu_return'][-1]
    print('The best model found is:', best_model)
    print('The best profit/loss for the investing period is: $%.2f' % best_return)
    if initial_capital != 0:
        print('The best return percentage for initial capital is: %.2f%%' % (best_return / initial_capital * 100))
    print()

    worst_model = None
    worst_return = long_only_return
    for key in portfolio_collection:
        if portfolio_collection[key]['accumu_return'][-1] < worst_return:
            worst_model = key
            worst_return = portfolio_collection[worst_model]['accumu_return'][-1]
    print('The worst model found is:', worst_model)
    print('The worst profit/loss for the investing period is: $%.2f' % worst_return)
    if initial_capital != 0:
        print('The worst return percentage for the initial capital is: %.2f%%' % (worst_return / initial_capital * 100))
    print()

    for key in model_collection:
        print('Processing portfolio for model:', key)
        trade_transactions = portfolio_collection[key][portfolio_collection[key].trade_action != 0]
        print(trade_transactions)
        print('Accumulated profit/loss for one share of stock with initial capital of $%.0f at the end of modeling period: $%.2f' % (initial_capital, portfolio_collection[key].accumu_return[-1]))
        if initial_capital != 0:
            return_percentage = portfolio_collection[key].accumu_return[-1] / initial_capital * 100
            print('Accumulated return percentage based on the initial capital investment: %.2f%%' % return_percentage)
        if trade_transactions.iloc[-1]['trade_action'] == 1:
            print('The current status of the model is:', 'Holding a position since', trade_transactions.index.tolist()[-1], '\n')
        else:
            print('The current status of the model is:', 'Waiting to enter since', trade_transactions.index.tolist()[-1], '\n')

# Example usage:
# task5_evaluate_performance(stock_quandl, model_collection, portfolio_collection, initial_capital)

## Task Execution and Output Management

In [ ]:
dataset_path = 'https://www.dainesanalytics.com/datasets/cramer-covid19-index/Cramer_COVID-19_Index.csv'
stock_meta = pd.read_csv(dataset_path, sep=',')
stock_list = stock_meta['Symbol'].tolist()
print('Stocks to process:', stock_list)

Stocks to process: ['AAPL', 'ABBV', 'ABT', 'ADBE', 'AKAM', 'AMD', 'AMT', 'AMZN', 'ATVI', 'BNTX', 'BSX', 'BYND', 'CAG', 'CCI', 'CHGG', 'CHWY', 'CL', 'CLX', 'CMG', 'CNC', 'COR', 'COST', 'COUP', 'CPB', 'CRM', 'CRWD', 'CTXS', 'D', 'DDOG', 'DG', 'DHR', 'DOCU', 'DPZ', 'DXCM', 'EA', 'EBAY', 'EBS', 'EQIX', 'ETSY', 'EVBG', 'FSLY', 'GILD', 'GIS', 'GOLD', 'GOOG', 'HD', 'HRL', 'JNJ', 'KR', 'LLY', 'LOGI', 'LVGO', 'MASI', 'MDLZ', 'MKC', 'MKTX', 'MRNA', 'MRVL', 'MSFT', 'NEM', 'NET', 'NFLX', 'NVDA', 'OKTA', 'PANW', 'PEP', 'PFE', 'PG', 'PLD', 'PRGO', 'PTON', 'PYPL', 'REGN', 'RMD', 'RNG', 'SHOP', 'SJM', 'SNY', 'SPGI', 'SPLK', 'SPOT', 'SQ', 'TDOC', 'TGT', 'TMO', 'TTD', 'TTWO', 'TW', 'TWLO', 'UNH', 'VEEV', 'VMW', 'VZ', 'WING', 'WIX', 'WMT', 'WORK', 'ZM', 'ZS', 'ZTS']


In [ ]:
import sys
from datetime import datetime

for stock_symbol in stock_list:
    # Begin the timer for the script processing
    startTimeModule = datetime.now()

    print(f'Processing {stock_symbol} from {model_start_date.strftime("%Y-%m-%d")} to {model_end_date.strftime("%Y-%m-%d")}')

    # Set up the redirection of output to a file
    orig_stdout = sys.stdout
    filename = f"{gdrivePrefix}algotrading_{stock_symbol}_ema-crossover_{model_end_date.strftime('%Y%m%d')}.txt"
    f = open('algotrading_AAPL_ema-crossover_20200623.txt', 'w')
    f = open('algotrading_AMZN_ema-crossover_20200623.txt', 'w')
    f = open('algotrading_GOOG_ema-crossover_20200623.txt', 'w')
    f = open('algotrading_NFLX_ema-crossover_20200623.txt', 'w')
    sys.stdout = f

    print(f'Processing the ticker symbol: {stock_symbol}')
    print(f"Starting date for the model: {model_start_date}")
    print(f"Ending date for the model: {model_end_date}")
    print()

try:
    stock_prices = task2_acquire_process_data(stock_symbol, stock_start_date, model_end_date, quandl_key)
    stock_models = task3_develop_strategy(stock_prices)
    stock_portfolios = task4_back_test_model(stock_models, initial_capital)
    task5_evaluate_performance(stock_prices, stock_models, stock_portfolios, initial_capital)
except IndexError as e:
    print(f"IndexError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

    sys.stdout = orig_stdout
    f.close()

    print(f'Total time for the script: {datetime.now() - startTimeModule}')
    print(f'The output was stored in the file: {filename}')

In [ ]:
print ('Total time for the script:',(datetime.now() - startTimeScript))